In [1]:
# imports
import numpy as np
import pandas as pd
import requests
import os 

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [116]:
"""RESTAURANT INFORMATION"""
# import data from previous part/defining variables
bikeDF = pd.read_csv('../data/Toronto_BikeCity.csv')

# creating variables for the request
FOURSQUARE_KEY = os.environ['FOURSQUARE_API_KEY']
url = "https://api.foursquare.com/v3/places/search"

# one list that will contain a bunch of dictionaries
FScontainerBARS = []

# loops through all the different bike stations, grabbing all the restaurants/bars within a 1000m radius
for index, row in bikeDF.iterrows():
    lat = row['Latitude']
    long = row['Longitude']

    # set parameters so it grabs specific information on bar/restuarants
    params = {
        'll': f'{lat},{long}',
        'categoryId': '13003,13065',  # ID for dining and drinking
        "radius": '1000',
        'fields': 'name,location,categories,distance,rating'
    }
    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_KEY
    }

    # gather the data and turn it into a JSON
    request = requests.get(url, params=params, headers=headers)


    data = request.json()

    # create a more refined JSON which goes into the 'results' key (this will make it easier to call specific elements later)
    resultsJSON = data['results']
    # turn the refined JSON into a dataframe(easier to grab info from)
    resultsDF = pd.DataFrame(resultsJSON)

    # #if the bar/restaurant has a rating, save it as a var
    # #if the bar/restaurant has no rating, set it a null/None
    try:
        resultsDF['rating']
    except:
        BarRating = 0
    else:
        BarRating = resultsDF['rating']


    # add a dictionary, containing all the relavent info, into the empty list
    # each entry in the list corresponds to a singular bike station
    FScontainerBARS.append({
        'Latitude': lat,
        'Longitude': long,
        'Rest/Bar Names': resultsDF['name'],
        'Distance': resultsDF['distance'],
        'Rest/ Bar Rating': BarRating
    })


Repeat process to get all gyms within a 1000m of each bike station

In [25]:
"""GYM INFORMATION"""

#one list that will contain a bunch of dictionaries
FScontainerGYMS =[]

#loops through all the different bike stations, grabbing all the gyms within a 1000m radius
for index, row in bikeDF.iterrows():
    lat = row['Latitude']
    long = row['Longitude']

    # set parameters so it grabs specific information on gyms
    params = {
        'll': f'{lat},{long}',
        'categories': 18021,  # ID for gyms and studios
        "radius": '1000',
        'fields': 'name,location,categories,distance'
    }
    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_KEY
    }

    #gather the data and turn it into a JSON
    request = requests.get(url, params=params, headers=headers)
    data=request.json()

    #create a more refined JSON which goes into the 'results' key (this will make it easier to call specific elements later)
    resultsJSON = data['results']
    #turn the refined JSON into a dataframe(easier to grab info from)
    resultsDF = pd.DataFrame(resultsJSON)
    


    #if there is a gym within a 1km radious, then save the names and distances
    #if there is no gym in the area, set name as null and distance as 0 (easier for calculations and spotting outliers later)
    if len(resultsJSON) > 0:
        GYMname =resultsDF['name']
        GYMdistance = resultsDF['distance']
    else:
        GYMname = None
        GYMdistance = 0,0 #double zero so we can use lambda later when calculating averages
    
    #add a dictionary, containing all the relavent info, into the empty list
    #each entry in the list corresponds to a singular bike station
    FScontainerGYMS.append({
            'Latitude': lat,
            'Longitude': long,
            'Gym Names': GYMname,
            'Distance': GYMdistance
            })


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
#requests and parsing of response had been completed in prior step/cell

Put your parsed results into a DataFrame

In [237]:
#Parsing the information into dataframes
FSbar_DF = pd.DataFrame(FScontainerBARS)
FSgym_DF = pd.DataFrame(FScontainerGYMS)

FSbar_DF
FSgym_DF

,Latitude,Longitude,Gym Names,Distance
0,43.665269,-79.319796,0 Beach Ballroom 1 Yoga ...,0 79 1 368 2 452 3 620 4 662 5...
1,43.671420,-79.445947,0 Planet Fitness 1...,0 504 1 467 2 554 3 581 4 634 5...
2,43.666224,-79.317693,0 Beach Ballroom 1 ...,0 270 1 456 2 556 3 598 4 602 5...
3,43.653236,-79.376716,0 Adelaide Club Toronto 1 ...,0 607 1 315 2 450 3 777 4 462 5...
4,43.663722,-79.380288,0 Canada's National Ballet School 1 ...,0 245 1 539 2 387 3 592 4 595 5...
...,...,...,...,...
712,43.601915,-79.499419,0 Mibody Health and Fitness Inc 1 ...,0 51 1 223 2 681 3 691 4 717 5...
713,43.642852,-79.531995,0 GoodLife Fitness Etobicoke Bloor Islingto...,0 838 1 130 2 278 3 330 4 9...
714,43.650945,-79.379498,0 Adelaide Club Toronto 1 ...,0 268 1 232 2 357 3 675 4 393 5...
715,43.707356,-79.437231,0 Bikran Yoga Forest Hill 1 ...,"0 1090 1 481 Name: distance, dtype: int64"


In [39]:
#Transforming data to get average distances
FSbar_DF['Average Distance_Bar'] = FSbar_DF['Distance'].apply(lambda x: sum(x)/len(x))
FSgym_DF['Average Distance_Gym'] = FSgym_DF['Distance'].apply(lambda x: sum(x)/len(x))

#left join 
FOURSQUARE_DF = pd.merge(FSbar_DF,FSgym_DF, on=['Latitude','Longitude'], how='left')

#cleaning data 
FOURSQUARE_DF.drop(columns=['Rest/Bar Names', 'Distance_x', 'Gym Names','Distance_y'], inplace=True)
FOURSQUARE_DF

,Latitude,Longitude,Rest/ Bar Rating,Average Distance_Bar,Average Distance_Gym
0,43.665269,-79.319796,0 8.9 1 8.9 2 7.5 3 7.8 4 7.5 5...,288.300000,624.700000
1,43.671420,-79.445947,0 8.4 1 6.7 2 6.2 3 9.1 4 7.6 5...,492.400000,675.400000
2,43.666224,-79.317693,0 7.8 1 7.5 2 8.9 3 7.7 4 8.0 5...,215.700000,598.100000
3,43.653236,-79.376716,0 9.1 1 8.3 2 8.9 3 8.6 4 8.8 5...,214.200000,574.200000
4,43.663722,-79.380288,0 8.3 1 8.2 2 8.3 3 7.8 4 8.8 5...,176.900000,484.300000
...,...,...,...,...,...
712,43.601915,-79.499419,0 8.9 1 6.7 2 6.8 3 7.8 4 8.3 5...,296.000000,531.428571
713,43.642852,-79.531995,0 7.9 1 7.4 2 6.7 3 6.6 4 6.6 5...,363.900000,1529.600000
714,43.650945,-79.379498,0 9.0 1 8.9 2 8.3 3 8.4 4 8.9 5...,158.900000,403.400000
715,43.707356,-79.437231,0 7.7 1 8.4 2 6.6 3 NaN 4 NaN 5...,519.600000,785.500000


In [ ]:
#Saving the dataframe as a .csv

#setting path
path = "../data/FOURSQUARE_DF.csv"

#saving
FOURSQUARE_DF.to_csv(path, index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [265]:
"""BARS AND RESTAURANTS"""
#creating variables for the request
YELP_KEY = os.environ['YELP_API_KEY']
url = "https://api.yelp.com/v3/businesses/search"

#one list that will contain a bunch of dictionaries
YELPcontainerBARS =[]

#loops through all the different bike stations, grabbing all the restaurants/bars within a 1000m radius
for index, row in bikeDF.iterrows():
    lat = row['Latitude']
    long = row['Longitude']

    params = {
        'latitude': lat,
        'longitude': long,
        'radius': '1000',
        'categories': 'restaurants,bars',
    }
    headers = {
        'Authorization': 'Bearer '+ YELP_KEY
    }

    response = requests.get(url, params=params, headers=headers)
    YELPdata = response.json()


    #create a more refined JSON which goes into the 'busisnesses' key (this will make it easier to call specific elements later)
    YELPJSON = YELPdata.get('businesses', [])
    
    #turn the refined JSON into a dataframe(easier to grab info from)
    resultsDF = pd.DataFrame(YELPJSON)

    # #if the bar/restaurant has a rating, save it as a var
    # #if the bar/restaurant has no rating, set it a null/None
    try:
        resultsDF['rating']
    except:
        BarRating = 0
    else:
        BarRating = resultsDF['rating']

    #add a dictionary, containing all the relavent info, into the empty list
    #each entry in the list corresponds to a singular bike station
    YELPcontainerBARS.append({
        'Latitude': lat,
        'Longitude': long,
        'Names':resultsDF['name'] ,
        'Distance': resultsDF['distance'],
        'REST/BAR RATING': BarRating
        })
    

KeyError: 'name'

In [263]:
"""GYM INFORMATION"""
#creating variables for the request

#one list that will contain a bunch of dictionaries
YELPcontainerGYM =[]

#loops through all the different bike stations, grabbing all the restaurants/bars within a 1000m radius
for index, row in bikeDF.iterrows():
    lat = row['Latitude']
    long = row['Longitude']

    params = {
        'latitude': lat,
        'longitude': long,
        'radius': '1000',
        'categories': 'gyms',
    }
    headers = {
        'Authorization': 'Bearer '+ YELP_KEY
    }

    response = requests.get(url, params=params, headers=headers)
    YELPdata = response.json()


    #create a more refined JSON which goes into the 'busisnesses' key (this will make it easier to call specific elements later)
    YELPJSON = YELPdata['businesses']

    #turn the refined JSON into a dataframe(easier to grab info from)
    resultsDF = pd.DataFrame(YELPJSON)


    #if there is a gym within a 1km radious, then save the names and distances
    #if there is no gym in the area, set name as null and distance as 0 (easier for calculations and spotting outliers later)
    if len(resultsJSON) > 0:
        GYMname =resultsDF['name']
        GYMdistance = resultsDF['distance']
    else:
        GYMname = None
        GYMdistance = 0,0 #double zero so we can use lambda later when calculating averages
    

    #add a dictionary, containing all the relavent info, into the empty list
    #each entry in the list corresponds to a singular bike station
    YELPcontainerGYM.append({
        'Latitude': lat,
        'Longitude': long,
        'Names': GYMname,
        'Distance': GYMdistance,
        })
    

KeyError: 'name'

In [256]:
os.environ['YELP_API_KEY'] = 'I3r6mf0yoKO41Z8z1YZO1xA8JK0lhCC9wC9deedgx2LKq8vAh56RfuM95mv9FyX3NQQJWZz5OA-bc8pu_RRbeogjZTUOCSyOEnYRPPLdKABmq63ObmeOTaDfAac1ZXYx'

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [262]:
YELP_KEY = os.environ['YELP_API_KEY']
url = "https://api.yelp.com/v3/businesses/search"

# one list that will contain a bunch of dictionaries
YELPcontainer = []

params = {
    'latitude': '37.77329',
    'longitude': '-122.4225',
    'radius': '1000',
    'categories': 'restaurants,bars',
}
headers = {
    'Authorization': 'Bearer ' + YELP_KEY
}

response = requests.get(url, params=params, headers=headers)
datajson = response.json()

yelp_results = datajson['businesses']
yelp_results

# frame = pd.DataFrame(datajson)
# frame

[{'id': 'ciEDsTWhajcdL3KuJqBRlw',
  'alias': 'espetus-churrascaria-san-francisco-2',
  'name': 'Espetus Churrascaria',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/F9G1pFFitfi9F4rJw_nrpQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/espetus-churrascaria-san-francisco-2?adjust_creative=Zz1LRyiQ_Yhi7ICiN83FzA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Zz1LRyiQ_Yhi7ICiN83FzA',
  'review_count': 3837,
  'categories': [{'alias': 'steak', 'title': 'Steakhouses'},
   {'alias': 'latin', 'title': 'Latin American'},
   {'alias': 'brazilian', 'title': 'Brazilian'}],
  'rating': 4.0,
  'coordinates': {'latitude': 37.7733327504928,
   'longitude': -122.422131667494},
  'transactions': ['restaurant_reservation', 'delivery'],
  'price': '$$$$',
  'location': {'address1': '1686 Market St',
   'address2': '',
   'address3': '',
   'city': 'San Francisco',
   'zip_code': '94102',
   'country': 'US',
   'state': 'CA',
   'display_address': ['1686 M

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [ ]:

names = []
ratings = []

station=0
while station < len(FSbar_DF['Rest/ Bar Rating']):
   # tmp = FSbar_DF['Rest/Bar Names'][station].to_list()
    names.append(FSbar_DF['Rest/Bar Names'][station].to_list()) 
    ratings.append(FSbar_DF['Rest/ Bar Rating'][station])
    station +=1
    
#list of all restaurants
NamesList= list(np.concatenate(names))
NamesList

flattened_ratings = []

for series in ratings:
    try:
        flattened_ratings.extend(series) # Attempt to iterate over the series
    except TypeError:
        flattened_ratings.append(series) # If TypeError occurs, treat the element as a single item

RESTAURANTSwithRATINGS = pd.DataFrame(NamesList, columns=['Restaurants'])
